In this assignment I will explore and cluster the neigbourhoods in Toronto. In this first assignment of this whole assignment the goal is to get the data from  Wikipedia page, transform it to a pandas dataframe and clean it so that the data is in a structured format. Lets begin.

In [4]:
# at first i import needed libraries 
import pandas as pd
import numpy as np


In [5]:
# then I wrangle the data from Wikibedia page
data_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_data = pd.read_html(data_path)


After I wrangled the data I checked in what format the data is (result is a list) and how many datasets does the data include (result was that the lenght of the dataset is 3). I also checked is the list on what format, and the result of this was a pandas Dataframe. 

So no I need to transform the first list to a dataframe called df_toronto_data. I also checked that does the data include duplicates in Postal Codes or Borough or Neighborhood - and the result is no. I will also clean the celss with borought such as Not assigned.

In [6]:
#trnasforming the data form first list into a Dataframe
df_toronto_data = toronto_data[0]

In [8]:
df_toronto_data = df_toronto_data.drop(df_toronto_data.loc[df_toronto_data["Borough"]=="Not assigned"].index, inplace=False).reset_index(drop = True)
df_toronto_data.head(12)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Created Dataframe consist three colums: PostalCode, Borough and Neighborhood
I have now ignored celss with a borought that is Not assingned.
In the data there is no twice listed PostalCodes - Neighborhoods which have the same PostalCode are mentioned in the same PostalCode (rows are combined into one row, separated with a comma)
Cells do not have Not assingned text either in Borough or in Neigborhood.


In [9]:
print(df_toronto_data.shape)

(103, 3)
